In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST, 
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)

IPython autoawait is `on`, and set to use `asyncio`
Subscribing too: connections


Listening webhooks failed! OSError(98, "error while attempting to bind on address ('0.0.0.0', 8022): address already in use") occurred.


In [7]:
response = await agent_controller.wallet.create_did()
print(response)
did = response["result"]["did"]
verkey = response["result"]["verkey"]

{'result': {'did': '3HUftQrUfFpcZ5VuGd5pog', 'verkey': '2FCryJ1X75Fy25FWhWbtyJxcFoNasZV3VggX1axCyikb', 'posture': 'wallet_only'}}


In [15]:
# write new DID to Sovrin Stagingnet
import requests
import json 
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

retry_strategy = Retry(
  total=3,
  backoff_factor=2
)
adapter = HTTPAdapter(max_retries=retry_strategy)
http = requests.Session()
http.mount("https://", adapter)
http.mount("http://", adapter)

url = 'http://localhost:9000/register-nym'

payload = {"role": "ENDORSER", "alias": "Justin","did": did, "verkey": verkey}

headers = {}
response = http.post(url, data=json.dumps(payload), headers=headers)


# url = 'https://localhost:9000/register'

# payload = {"role": "ENDORSER", "alias": "Justin", "seed": "213", "did": "null"}

# # Adding empty header as parameters are being sent in payload
# headers = {}


# r = requests.post(url, data=json.dumps(payload), headers=headers, verify=False)
print(response.json())
print(response.status_code)



ConnectionError: HTTPConnectionPool(host='localhost', port=9000): Max retries exceeded with url: /register-nym (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0xffff7bedd510>: Failed to establish a new connection: [Errno 111] Connection refused'))